# Web scraping, Stage 2

Improving Google Scholar search with alternative keywords.

Import needed libraries.

In [1]:
import numpy as np
import pandas as pd
import json
from scholarly import scholarly, ProxyGenerator
from scholarly._navigator import MaxTriesExceededException

Make a list of the university names.

In [2]:
universities = ['oulu',
               'bochum',
               'porto',
               'bordeaux',
               'lodz']

## Re-query failed authors

We have created a file containing indices of authors in the Excel spreadsheet that will be re-queried with the scholarly API.

We load them into a dictionary.

In [3]:
idx_to_search = {}
with open('failed_for_search.txt') as f:
    for l in f.readlines():
        l = l.split(',')
        idx_to_search[l[0]] = [int(i) for i in l[1:]]

Let's see what we've made.

In [4]:
idx_to_search

{'oulu': [3, 8, 45, 46, 54, 55, 58, 71, 92],
 'bochum': [15, 17, 23, 44, 49],
 'porto': [1, 23, 46, 47, 57, 60, 68, 87, 94],
 'bordeaux': [12, 16, 22, 27, 38, 55],
 'lodz': [8, 15, 24, 36, 52, 54, 59, 74, 75, 80, 85]}

We read our new, extended and modified spreadsheet with extra `Alternative name`s to search for.

In [5]:
dfs = pd.read_excel('staff.xlsx', None)
dfs = dict((universities[i], value) for ((key, value), i) in zip(dfs.items(), range(5)))
for df in dfs.values():
    df['Gender'].replace({'Male': 'M', 'Female': 'F'}, inplace=True)
    if not 'Full name' in df:
        df['Full name'] = df['First name'].str.cat(df['Last name'], sep=' ')

Let's verify the indices we have specified are the correct ones by checking the loaded spreadsheet and also filtering. 

Seems to be OK.

In [6]:
for uni_name in universities:
    dfs[uni_name] = dfs[uni_name].iloc[idx_to_search[uni_name]].reset_index(drop=True)
    print(dfs[uni_name])

    First name   Last name Gender                 Role  \
0         Sasu     Tarkoma      M            Professor   
1        Anna     Yershova      F  University Lecturer   
2        Heli    Koskimäki      F    ADjunct Professor   
3  Hannu-Pekka       Komsa      M  Assistant Professor   
4      Glisic         Savo      M            Professor   
5          Tad  Matsumoto       M            Professor   
6       Marian    Codreanu      M    ADjunct Professor   
7          Li          Liu      F  Assistant Professor   
8        Marja    Harjumaa      F    Adjunct Professor   

           Research Group                        Alternative search  \
0    Ubiquitous Computing                    University of Helsinki   
1    Ubiquitous Computing                                       NaN   
2                    BISG                               Oura Health   
3                     MIC                           alto university   
4                     CWC                                       

We are going to re-use the function we specified in the previous notebook.

In [7]:
def retrieve_authors(df, uni_name):
    authors = []
    failed = []

    num_authors = len(df)
    for i, name in enumerate(df['Full name']):
        if i % 10 == 0:
            print('Retrieving author no. {} out of {} for University of {}'.format(i + 1,
                                                                                   num_authors, 
                                                                                   uni_name.capitalize()))
        if 'Alternative search' in df and not pd.isnull(df['Alternative search'][i]):
            if df['Alternative search'][i] == '<blank>':
                query = scholarly.search_author(name)
            else:
                query = scholarly.search_author(name + ', ' + df['Alternative search'][i])
        else:
            query = scholarly.search_author(name + ', ' + uni_name)
        try:
            try:
                author = scholarly.fill(next(query), sections=['coauthors'])
            except MaxTriesExceededException:
                print('Query failed. Generating new proxy')
                pg = ProxyGenerator()
                pg.FreeProxies()
                scholarly.use_proxy(pg)
                author = scholarly.fill(next(query), sections=['coauthors'])
            
            author_dict = {}
            author_dict['scholar_id']  = author['scholar_id']
            author_dict['name'] = author['name']
            author_dict['affiliation'] = author['affiliation']
            author_dict['gender'] = df['Gender'][i]
            author_dict['role'] = df['Role'][i]
            author_dict['url_picture'] = author['url_picture']
            author_dict['coauthors'] = [coauth['scholar_id'] for coauth in author['coauthors']]
            authors.append(author_dict)
        except StopIteration:
            failed.append(name)

    print('Number of retrieved authors:', len(authors))
    print('Number of failed authors:', len(failed))
    
    return authors, failed

If all goes well, there should be no failed authors.

In [8]:
requeried_authors = []
for uni_name, df in dfs.items():
    authors, failed = retrieve_authors(df, uni_name)
    requeried_authors.append({'university': uni_name, 'authors': authors, 'failed': failed})

Retrieving author no. 1 out of 9 for University of Oulu
Number of retrieved authors: 9
Number of failed authors: 0
Retrieving author no. 1 out of 5 for University of Bochum
Number of retrieved authors: 5
Number of failed authors: 0
Retrieving author no. 1 out of 9 for University of Porto
Number of retrieved authors: 9
Number of failed authors: 0
Retrieving author no. 1 out of 6 for University of Bordeaux
Number of retrieved authors: 6
Number of failed authors: 0
Retrieving author no. 1 out of 11 for University of Lodz
Retrieving author no. 11 out of 11 for University of Lodz
Number of retrieved authors: 11
Number of failed authors: 0


## Merge successful and re-queried authors

We retrieve the queried authors from stage 1.

In [9]:
with open('../Stage 1/uni_authors.json') as f:
    uni_authors = json.load(f)

Let's make sure the university names align.

In [10]:
[authors['university'] for authors in requeried_authors] == [authors['university'] 
                                                             for authors in uni_authors]

True

How many are the successfully queried and failed authors __before__ merging authors and removing requeried authors from the failed list?

In [11]:
for authors in uni_authors:
    print('Successful authors for University of {}:'.format(authors['university'].capitalize()), 
          len(authors['authors']))
    print('Failed authors for University of {}:'.format(authors['university'].capitalize()), 
          len(authors['failed']))

Successful authors for University of Oulu: 78
Failed authors for University of Oulu: 38
Successful authors for University of Bochum: 31
Failed authors for University of Bochum: 23
Successful authors for University of Porto: 66
Failed authors for University of Porto: 37
Successful authors for University of Bordeaux: 30
Failed authors for University of Bordeaux: 35
Successful authors for University of Lodz: 40
Failed authors for University of Lodz: 57


Append requeried authors to the successful authors.

In [12]:
for i, authors in enumerate(uni_authors):
    scholarIDs = map(lambda x: x['scholar_id'], authors['authors'])
    for ra in requeried_authors[i]['authors']:
        if ra['scholar_id'] not in scholarIDs:
            authors['authors'].append(ra)

## Remove requeried authors from failed authors

For this task, we are going to load the previous version of the spreadsheet.

In [13]:
dfs2 = pd.read_excel('../Stage 1/staff.xlsx', None)
dfs2 = dict((universities[i], value) for ((key, value), i) in zip(dfs2.items(), range(5)))
for df in dfs2.values():
    df['Gender'].replace({'Male': 'M', 'Female': 'F'}, inplace=True)
    if not 'Full name' in df:
        df['Full name'] = df['First name'].str.cat(df['Last name'], sep=' ')

If all goes well, there should be only one `Alternative search` column for the loaded DataFrames,
in the porto DataFrame.

In [14]:
dfs2

{'oulu':      First name    Last name Gender                 Role  \
 0    Steven M.       LaValle      M            Professor   
 1         Timo         Ojala      M            Professor   
 2        Jukka        Riekki      M            Professor   
 3          Sasu      Tarkoma      M            Professor   
 4       Denzil      Ferreira      M  Associate Professor   
 ..          ...          ...    ...                  ...   
 111   Alexander       Bykov       M    Adjunct Professor   
 112        Esko  Alasaarela       M            Professor   
 113  Christian        Schuss      M  University Lecturer   
 114       Risto     Myllylä       M            Professor   
 115        Igor   Meglinski       M            Professor   
 
            Research Group           Full name  
 0    Ubiquitous Computing  Steven M.  LaValle  
 1    Ubiquitous Computing         Timo  Ojala  
 2    Ubiquitous Computing       Jukka  Riekki  
 3    Ubiquitous Computing        Sasu Tarkoma  
 4    Ubiquit

We have created another file containing indices of authors for the spreadsheet from stage 1 that were
re-queried due to slight differences between the spreadsheets.

We load them into another dictionary.

In [15]:
idx_to_search_s1 = {}
with open('failed_for_search_stage1.txt') as f:
    for l in f.readlines():
        l = l.split(',')
        idx_to_search_s1[l[0]] = [int(i) for i in l[1:]]

Let's verify the indices are the correct ones by comparing the spreadsheets while filtering in the meantime.

Seems to be OK.

In [16]:
for uni_name in universities:
    dfs2[uni_name] = dfs2[uni_name].iloc[idx_to_search_s1[uni_name]]
    print(dfs2[uni_name])

     First name   Last name Gender                 Role  \
3          Sasu     Tarkoma      M            Professor   
8         Anna      LaValle      F  University Lecturer   
45        Heli    Koskimäki      F    ADjunct Professor   
46  Hannu-Pekka       Komsa      M  Assistant Professor   
54      Glisic         Savo      M            Professor   
55      Tadashi  Matsumoto       M            Professor   
58       Marian    Codreanu      M    ADjunct Professor   
71          Li          Liu      F  Assistant Professor   
92        Marja    Harjumaa      F    Adjunct Professor   

            Research Group           Full name  
3     Ubiquitous Computing        Sasu Tarkoma  
8     Ubiquitous Computing       Anna  LaValle  
45                    BISG     Heli  Koskimäki  
46                     MIC   Hannu-Pekka Komsa  
54                     CWC        Glisic  Savo  
55                     CWC  Tadashi Matsumoto   
58  CWC-Radio Technologies     Marian Codreanu  
71               

We create a dictionary of the requeried authors for the spreadsheet from stage 1.

The dictionary should contain only Series objects.

In [17]:
dfs2_failed_names = {}
for i in dfs2:
    dfs2_failed_names[i] = dfs2[i]['Full name']

If we've done everything well so far, removing the names of the requeried authors from the failed authors of the data produced in stage 1 should produce no error, with the exception of porto authors.

In [18]:
for authors in uni_authors:
    for name in dfs2_failed_names[authors['university']]:
        try:
            authors['failed'].remove(name)
        except ValueError:
            print(authors['university'] + ',', name)

porto, Adriano Carvalho
porto, Helder Leite
porto, Cândido Duarte
porto, Maria do Rosário de Pinho
porto, Nuno Cruz
porto, Paulo Portugal
porto, António Porto
porto, Hugo Pacheco
porto, Pedro Ribeiro


Let's check again the number of successfully queried and failed authors for each university. Our data should now be changed.

In [19]:
for authors in uni_authors:
    print('Successful authors for University of {}:'.format(authors['university'].capitalize()), 
          len(authors['authors']))
    print('Failed authors for University of {}:'.format(authors['university'].capitalize()), 
          len(authors['failed']))

Successful authors for University of Oulu: 87
Failed authors for University of Oulu: 29
Successful authors for University of Bochum: 36
Failed authors for University of Bochum: 18
Successful authors for University of Porto: 66
Failed authors for University of Porto: 37
Successful authors for University of Bordeaux: 36
Failed authors for University of Bordeaux: 29
Successful authors for University of Lodz: 51
Failed authors for University of Lodz: 46


## Save new author data

We dump all the new author information into a JSON file.

In [20]:
with open('uni_authors.json', 'w') as f:
    json.dump(uni_authors, f)

And also dump the failed author names into another, separate JSON file.

In [21]:
uni_authors_failed = [{'university': authors['university'], 
                       'failed': authors['failed']} for authors in uni_authors]

with open('uni_authors_failed.json', 'w') as f:
    json.dump(uni_authors_failed, f)